In [144]:
import copy
import json
import random

In [169]:
blocks_f = 'data/blocks.json'
with open(blocks_f) as f:
    blocks = json.load(f)

students_f = 'data/priorities.json'
with open(students_f) as f:
    students_tmp = json.load(f)

N_blocks = len(blocks)

for block in blocks.values():
    for module, slots in block.items():
        block[module] = {'slots': slots, 'prio1': 0, 'prio2': 0, 'prio3': 0, 'factor': 1, 'IDs': []}

students = []

for i, student in enumerate(students_tmp):
    
    s = {'ID': f'{i:03}',
        'last_name': student['last_name'],
        'first_name': student['first_name'],
        'class': student['class'],
        'block1': {'prio1': [], 'prio2': [], 'prio3': [], 'weight': 1},
        'block2': {'prio1': [], 'prio2': [], 'prio3': [], 'weight': 1},
        'block3': {'prio1': [], 'prio2': [], 'prio3': [], 'weight': 1},
        'block4': {'prio1': [], 'prio2': [], 'prio3': [], 'weight': 1},
        'block5': {'prio1': [], 'prio2': [], 'prio3': [], 'weight': 1}
    }
    for p in student['priorities']:
        if (mod_prio := p['priority']) in (1, 2, 3):
            
            mod_type = p['module_type']
            mod_name = p['module_name']
            s['block'+str(mod_type)]['prio'+str(4-mod_prio)].append(mod_name)

            blocks[str(mod_type)][mod_name]['prio'+str(4-mod_prio)] += 1

    students.append(s)

In [170]:
for block in blocks.values():
    for module in block:
        slots = block[module]['slots']
        prio1 = block[module]['prio1']
        prio2 = block[module]['prio2']
        prio3 = block[module]['prio3']

        if (empty := slots-prio1) >= 0:
            block[module]['factor'] += 1
            if (empty := empty-prio2) >= 0:
                block[module]['factor'] += 1
                if empty - prio3 >= 0:
                    block[module]['factor'] += 1


In [171]:
for student in students:
    for i in range(N_blocks):
        s = student['block'+str(i+1)]
        s['weight'] = len(s['prio1']) + len(s['prio2']) + len(s['prio3'])

In [201]:
try_blocks = copy.deepcopy(blocks)

for i, block in enumerate(try_blocks.values(), start=1):
    block_key = 'block'+str(i)
    students = sorted(students, key=lambda x: x[block_key]['weight']+random.random(), reverse=True)

    for student in students:
        block_data = student[block_key]
        prio1 = block_data['prio1']
        prio2 = block_data['prio2']
        prio3 = block_data['prio3']
        options = []
        for mod_key, mod_value in block.items():
            if mod_value['slots'] != 0:
                opt_factor = mod_value['factor']
                if mod_key in prio1:
                    opt_factor *= 30
                elif mod_key in prio2:
                    opt_factor *= 20
                elif mod_key in prio3:
                    opt_factor *= 10
            
                options += [mod_key] * opt_factor

        choice = random.choice(options)
        block_data['choice'] = choice
        block[choice]['slots'] -= 1
        block[choice]['IDs'].append(student['ID'])



In [202]:
block_num = 5
total = 0

for mod_key, mod_val in try_blocks[str(block_num)].items():
    ids = mod_val['IDs']
    slots = mod_val['slots']
    p1 = mod_val['prio1']
    p2 = mod_val['prio2']
    p3 = mod_val['prio3']
    prio1 = prio2 = prio3 = prio0 = 0
    for id in ids:
        student = next(s for s in students if s['ID'] == id)
        if mod_key in student['block'+str(block_num)]['prio1']:
            prio1 += 1
        elif mod_key in student['block'+str(block_num)]['prio2']:
            prio2 += 1
        elif mod_key in student['block'+str(block_num)]['prio3']:
            prio3 += 1
        else:
            prio0 += 1
        
    print(f'{mod_key}, {slots}, {prio1} ({p1}), {prio2} ({p2}), {prio3} ({p3}), {prio0}')
    total += prio1+prio2+prio3+prio0

print(total)


Oberflächen und Farben, 0, 2 (6), 12 (30), 1 (33), 5
Fourier-Reihen, 8, 4 (6), 4 (16), 1 (20), 3
Datenexploration, 12, 2 (11), 2 (19), 1 (22), 3
Astrophysik, Planetologie und Raumfahrt, 3, 9 (31), 6 (44), 1 (27), 1
Molekulare Schalter – eine on/off-Beziehung, 0, 0 (6), 11 (39), 0 (21), 1
Let's Arduino, 4, 0 (4), 3 (21), 1 (23), 4
Die Hoffnung ist erneuerbar, 11, 1 (3), 3 (23), 3 (30), 2
Swiss Young Physicists' Tournament, 10, 5 (10), 1 (4), 1 (15), 3
Von Big Data und Künstlicher Intelligenz zur Umweltinformatik, 3, 3 (8), 6 (30), 3 (24), 1
Woher kommst du wirklich? Frag deine DNA!, 4, 11 (51), 4 (47), 1 (16), 0
Astronomie, 0, 5 (12), 12 (30), 2 (33), 1
Entsorgung, Recycling und Abwasserreinigung in Zürich, 9, 2 (2), 2 (13), 1 (24), 2
Praktische Molekulargenetik: Wolbachia – auf der Suche nach Bakterien-DNA in selbstgefangenen Insekt, 0, 4 (10), 7 (36), 0 (19), 3
Energiespeicherung – Vom Akku bis zum Pump-Speicherstausee, 8, 5 (9), 1 (19), 1 (38), 5
178


In [188]:
len(students)

178